In [1]:
import eospy
from eospy.cleos import Cleos
import pytz
import random
import datetime
import time
import csv
import numpy as np

import pandas as pd

ce = Cleos(url='https://api.testnet.eos.io')
info = ce.get_info()

In [2]:
print(info)

{'server_version': 'b39b6bd3', 'chain_id': 'cf057bbfb72640471fd910bcb67639c22df9f92470936cddc1ade0e2f2e7dc4f', 'head_block_num': 6432660, 'last_irreversible_block_num': 6432330, 'last_irreversible_block_id': '0062264a68de29b770e3ea5a143dcb1b2cbcc1d081dc017bafb5578862b9f6ba', 'head_block_id': '006227949f4f2eae45e6c0355a87dec6e0f1b9fc801298e25d8e9bdabd5bd4d2', 'head_block_time': '2020-05-30T11:25:56.500', 'head_block_producer': 'initn', 'virtual_block_cpu_limit': 200000000, 'virtual_block_net_limit': 1048576000, 'block_cpu_limit': 199900, 'block_net_limit': 1048576, 'server_version_string': 'v2.0.4', 'fork_db_head_block_num': 6432660, 'fork_db_head_block_id': '006227949f4f2eae45e6c0355a87dec6e0f1b9fc801298e25d8e9bdabd5bd4d2', 'server_full_version_string': 'v2.0.4-b39b6bd3c74ff47fc53fa80aafa2c1941ba82879'}


In [3]:
data_index = ['txid', 'customer', 'temp_cus', 'time_c', 'r_w', 'temp_rw', 'time_rw', 'r_w', 'temp_rw', 'time_rw', 'de', 'temp_de', 'time_de']

In [4]:
format(datetime.datetime.now())[:-7]

'2020-05-30 19:25:56'

In [5]:
data_save = []

In [6]:
class block_pusher():
    def __init__(self, endpoint, contract_account):
        self.ce = Cleos(url=endpoint)
        self.contract_account = contract_account
        
        self.user_list = []
        self.restaurant_list = []
        self.deliver_man_list = []
        
        self.tasks = []
        
    def add_user(self, account_name, account_active_private_key):
        self.user_list.append([account_name, account_active_private_key])
        
    def add_restaurant(self, account_name, account_active_private_key):
        self.restaurant_list.append([account_name, account_active_private_key])
        
    def add_deliver_man(self, account_name, account_active_private_key):
        self.deliver_man_list.append([account_name, account_active_private_key])
        
    def create_tx_id(self):
#         id_map = '12345abcdefghijklmnopqrstuvwxyz'
#         id_tx = ''
#         for i in range(10):
#             sele = random.randint(0,len(id_map)-1)
#             id_tx = id_tx + id_map[sele]
#         return id_tx
        return format(random.randint(10000000,99999999))
        
    def create_tasks(self, mount_out_task = 50):
        self.tasks = []
        for i in range(mount_out_task):
            
            tx_id = self.create_tx_id()
            
            user = random.randint(0,len(pub.user_list)-1)
            restaurant = random.randint(0,len(pub.restaurant_list)-1)
            deliver_man = random.randint(0,len(pub.deliver_man_list)-1)
            
            user_temp = 36+random.randint(0,15)/10
            cook_temp = 36+random.randint(0,15)/10
            deliver_temp = 36+random.randint(0,15)/10
            
            self.tasks.append([tx_id, user, restaurant, deliver_man, user_temp, cook_temp, deliver_temp])
            
    def push_contract(self, action , tx_id, account, account_key, temperture):
        ##################################################################################################################
        if (action in 'c2r'):
            arguments = {
                "txid": tx_id,  # sender
                "customer": account,  # receiver
                "temp_cus": format(temperture),  # In EOS
                "time_c": format(datetime.datetime.now())[:-7],
            }
        elif (action in 'rc2rw'):
            arguments = {
                "txid": tx_id,  # sender
                "r_cook": account,  # receiver
                "temp_rc": format(temperture),  # In EOS
                "time_rc": format(datetime.datetime.now())[:-7],
            }
        elif (action in 'rw2de'):
            arguments = {
                "txid": tx_id,  # sender
                "r_w": account,  # receiver
                "temp_rw": format(temperture),  # In EOS
                "time_rw": format(datetime.datetime.now())[:-7],
            }
        elif (action in 'de2c'):
            arguments = {
                "txid": tx_id,  # sender
                "de": account,  # receiver
                "temp_de": format(temperture),  # In EOS
                "time_de": format(datetime.datetime.now())[:-7],
            }
        ##################################################################################################################
        print("arg = ",arguments)
        payload = {
            "account": self.contract_account,
            "name": action,
            "authorization": [{
                "actor": account,
                "permission": "active",
            }],
        }
        print("pay = ",payload)
        #print("key = ",account_key)
        #Converting payload to binary
        data = ce.abi_json_to_bin(payload['account'], payload['name'], arguments)
        #Inserting payload binary form as "data" field in original payload
        payload['data'] = data['binargs']
        #final transaction formed
        trx = {"actions": [payload]}
        import datetime as dt
        trx['expiration'] = str(
            (dt.datetime.utcnow() + dt.timedelta(seconds=60)).replace(tzinfo=pytz.UTC))
        # use a string or EOSKey for push_transaction
        # key = "5KQwrPbwdL6PhXujxW37FSSQZ1JiwsST4cqQzDeyXtP79zkvFD3"
        # use EOSKey:
        key = eospy.keys.EOSKey(account_key)
        resp = ce.push_transaction(trx, key, broadcast=True)
        print('------------------------------------------------')
        #print(resp)
        #print('------------------------------------------------')
        
    def push_c2r(self, tx_id, index, user_temp):
        self.push_contract('c2r', tx_id, self.user_list[index][0], self.user_list[index][1],user_temp)
        
    def push_rc2rw(self, tx_id, index, restaurant_temp):
        self.push_contract('rc2rw', tx_id, self.restaurant_list[index][0], self.restaurant_list[index][1], restaurant_temp)
        
    def push_rw2de(self, tx_id, index, restaurant_temp):
        # a little be different, this also push restaurant's account
        self.push_contract('rw2de', tx_id, self.restaurant_list[index][0], self.restaurant_list[index][1], restaurant_temp)
    
    def push_de2c(self, tx_id, index, deliver_man_temp):
        # a little be different, this push  delivery mam's account
        self.push_contract('de2c', tx_id, self.deliver_man_list[index][0], self.deliver_man_list[index][1], deliver_man_temp)

    def run_tasks(self):
        #    0.     1.      2.          3.           4.        5.            6. 
        # [tx_id, user, restaurant, deliver_man, user_temp, cook_temp, deliver_temp]
        
        for i in range(len(self.tasks)):
            test_time = 0
#             print("push de2c")
#             self.push_de2c(self.tasks[i][0], self.tasks[i][3], self.tasks[i][4])
            while True:
                try:
                    test_time += 1
                    print("push cr2")
                    self.push_c2r(self.tasks[i][0], self.tasks[i][1], self.tasks[i][4])
                    time.sleep(2)

                    print("push rc2rw")
                    self.push_rc2rw(self.tasks[i][0], self.tasks[i][2], self.tasks[i][5])
                    time.sleep(2)

                    print("push rw2de")
                    self.push_rw2de(self.tasks[i][0], self.tasks[i][2], self.tasks[i][5])
                    time.sleep(2)

                    print("push de2c")
                    self.push_de2c(self.tasks[i][0], self.tasks[i][3], self.tasks[i][6])
                    time.sleep(2)
                except:
                    if(test_time<5):
                        print(">>AGAIN<<")
                        continue
                    print("ABORT")
                break
            print("[TASK : ",i+1,"/",len(self.tasks)," done]")

        

In [7]:
pub = block_pusher('https://api.testnet.eos.io', 'ozjkloygsxzk')

In [8]:
with open('/root/notebooks/final/eos_keys.txt', newline='') as f:
    reader = csv.reader(f)
    data = list(reader)

In [9]:

pub.add_user(data[0][0],data[0][1])
pub.add_user(data[1][0],data[1][1])
pub.add_user(data[2][0],data[2][1])
pub.add_user(data[3][0],data[3][1])
pub.add_user(data[4][0],data[4][1])
pub.add_user(data[5][0],data[5][1])
pub.add_user(data[6][0],data[6][1])
pub.add_user(data[7][0],data[7][1])

# pub.add_user("123","123")
# pub.add_user("123","123")

pub.add_restaurant(data[8][0],data[8][1])
pub.add_restaurant(data[9][0],data[9][1])
pub.add_restaurant(data[10][0],data[10][1])
pub.add_restaurant(data[11][0],data[11][1])
# pub.add_restaurant("123","123")
# pub.add_restaurant("123","123")

pub.add_deliver_man(data[12][0],data[12][1])
pub.add_deliver_man(data[13][0],data[13][1])
pub.add_deliver_man(data[14][0],data[14][1])
pub.add_deliver_man(data[15][0],data[15][1])
pub.add_deliver_man(data[16][0],data[16][1])
pub.add_deliver_man(data[17][0],data[17][1])
pub.add_deliver_man(data[18][0],data[18][1])
pub.add_deliver_man(data[19][0],data[19][1])
pub.add_deliver_man(data[20][0],data[20][1])
pub.add_deliver_man(data[21][0],data[21][1])
pub.add_deliver_man(data[22][0],data[22][1])
pub.add_deliver_man(data[23][0],data[23][1])
# pub.add_deliver_man("123","123")
# pub.add_deliver_man("123","123")


In [10]:
pub.create_tasks(10)

In [11]:
pub.tasks

[['72251699', 3, 0, 6, 36.7, 36.1, 36.2],
 ['17994248', 3, 3, 9, 36.2, 37.3, 36.6],
 ['66808893', 2, 1, 7, 36.0, 36.4, 36.2],
 ['89903591', 5, 0, 2, 37.2, 37.0, 37.4],
 ['93912590', 0, 2, 10, 36.4, 37.2, 36.9],
 ['48720094', 1, 1, 2, 37.3, 37.4, 36.1],
 ['92707776', 2, 3, 10, 36.2, 36.9, 36.9],
 ['62093289', 2, 1, 0, 36.4, 36.5, 36.7],
 ['80592183', 1, 2, 11, 37.2, 36.6, 37.2],
 ['90432220', 0, 1, 1, 36.9, 36.3, 37.5]]

In [12]:
now = format(datetime.datetime.now())[:-7]

In [13]:
pub.run_tasks()

push cr2
arg =  {'txid': '72251699', 'customer': 'tdmdwfbyrieq', 'temp_cus': '36.7', 'time_c': '2020-05-30 19:25:57'}
pay =  {'account': 'ozjkloygsxzk', 'name': 'c2r', 'authorization': [{'actor': 'tdmdwfbyrieq', 'permission': 'active'}]}
------------------------------------------------
push rc2rw
arg =  {'txid': '72251699', 'r_cook': 'nrcxmhvmmdws', 'temp_rc': '36.1', 'time_rc': '2020-05-30 19:26:00'}
pay =  {'account': 'ozjkloygsxzk', 'name': 'rc2rw', 'authorization': [{'actor': 'nrcxmhvmmdws', 'permission': 'active'}]}
------------------------------------------------
push rw2de
arg =  {'txid': '72251699', 'r_w': 'nrcxmhvmmdws', 'temp_rw': '36.1', 'time_rw': '2020-05-30 19:26:03'}
pay =  {'account': 'ozjkloygsxzk', 'name': 'rw2de', 'authorization': [{'actor': 'nrcxmhvmmdws', 'permission': 'active'}]}
------------------------------------------------
push de2c
arg =  {'txid': '72251699', 'de': 'cqttboxcmqbg', 'temp_de': '36.2', 'time_de': '2020-05-30 19:26:07'}
pay =  {'account': 'ozjkl

In [14]:
get_table = ce.get_table('ozjkloygsxzk','ozjkloygsxzk', 'people',lower_bound='0',limit=-1)

In [15]:
data_save = []
for i in get_table['rows']:
    data = []
    for k in data_index:
        data.append(i[k])
    data_save.append(data)

In [16]:
df = pd.DataFrame(data_save)
df_new = df.rename(columns=lambda s: data_index[s])

In [17]:
for i in data_index:
    df_new[i] = df_new[i].replace('', np.nan)
df_new = df_new.dropna()

In [21]:
now_ = format(datetime.datetime.now())[:-7]

lower_bound = df_new['time_c'] > now
upper_bound = df_new['time_c'] < now_

print("Time range:")
print("From : ",now)
print("TO   : ",now_)

Time range:
From :  2020-05-30 19:25:56
TO   :  2020-05-30 19:29:29


In [19]:
df_new = df_new.sort_values(by=['time_c'])
df_new = df_new[lower_bound & upper_bound]

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [20]:
df_new

,txid,customer,temp_cus,time_c,r_w,temp_rw,time_rw,r_w,temp_rw,time_rw,de,temp_de,time_de
74,72251699,tdmdwfbyrieq,36.7,2020-05-30 19:25:57,nrcxmhvmmdws,36.1,2020-05-30 19:26:03,nrcxmhvmmdws,36.1,2020-05-30 19:26:03,cqttboxcmqbg,36.2,2020-05-30 19:26:07
16,17994248,tdmdwfbyrieq,36.2,2020-05-30 19:26:10,mflhbiwckvye,37.3,2020-05-30 19:26:18,mflhbiwckvye,37.3,2020-05-30 19:26:18,bogidqzffpoq,36.6,2020-05-30 19:26:22
67,66808893,fbxruqcaetsa,36.0,2020-05-30 19:26:26,tavwwqactbew,36.4,2020-05-30 19:26:33,tavwwqactbew,36.4,2020-05-30 19:26:33,azdyojobmkea,36.2,2020-05-30 19:26:36
95,89903591,xmlllqshzxgq,37.2,2020-05-30 19:26:40,nrcxmhvmmdws,37.0,2020-05-30 19:26:47,nrcxmhvmmdws,37.0,2020-05-30 19:26:47,jmgpjsfyhqmc,37.4,2020-05-30 19:26:51
101,93912590,vdqfllefkglk,36.4,2020-05-30 19:26:55,cslsovhydlks,37.2,2020-05-30 19:27:04,cslsovhydlks,37.2,2020-05-30 19:27:04,wtvpjqkffkem,36.9,2020-05-30 19:27:09
46,48720094,nzquzuexsbqw,37.3,2020-05-30 19:27:12,tavwwqactbew,37.4,2020-05-30 19:27:19,tavwwqactbew,37.4,2020-05-30 19:27:19,jmgpjsfyhqmc,36.1,2020-05-30 19:27:23
99,92707776,fbxruqcaetsa,36.2,2020-05-30 19:27:26,mflhbiwckvye,36.9,2020-05-30 19:27:34,mflhbiwckvye,36.9,2020-05-30 19:27:34,wtvpjqkffkem,36.9,2020-05-30 19:27:37
57,62093289,fbxruqcaetsa,36.4,2020-05-30 19:27:41,tavwwqactbew,36.5,2020-05-30 19:27:49,tavwwqactbew,36.5,2020-05-30 19:27:49,bhmtdvrjksmm,36.7,2020-05-30 19:27:53
85,80592183,nzquzuexsbqw,37.2,2020-05-30 19:27:56,cslsovhydlks,36.6,2020-05-30 19:28:04,cslsovhydlks,36.6,2020-05-30 19:28:04,izwatdoooomk,37.2,2020-05-30 19:28:08
98,90432220,vdqfllefkglk,36.9,2020-05-30 19:28:11,tavwwqactbew,36.3,2020-05-30 19:28:20,tavwwqactbew,36.3,2020-05-30 19:28:20,azvaphsayems,37.5,2020-05-30 19:28:24
